In [71]:
#https://blog.dataiku.com/2016/10/08/machine-learning-markov-chains-generate-clinton-trump-quotes
#TODO: get a lot more text for each author, 
#implement starting a phrase with a <sentence begin> and ending it with a <sentence end>
#implement Backoff, 
#implement word weights (right now the ngram densities are too low for weights to do much) for overall and in-dialogue
#graph search
#A state is the N words at the end of the sentence, and the list of magic words. A transition between states is the adding of a new word to the end of res. 
#Ngrams alg defines neighbors
import nltk
import random

In [168]:
N = 3
MAX_DIALOGUE_LEN = 15
AUTHOR_AUG = "Augistine"
AUTHOR_ZZ = "Zhuangzi"
MAGIC_WORDS = set(["god","spirit","way","death","heaven","life","meaning","hell","water"])
grams1 = {}
grams2 = {}

In [169]:
def initialize_grams(grams, textfile): #initialize the dicts that store the markov probabilities for N-word chains
    f = open(textfile)
    s = f.read().replace('(','').replace(')','')
    t = nltk.word_tokenize(s)
    for i in range(len(t)-(N)):
        trigram = tuple(t[x].lower() for x in range(i,i+N))
        if trigram in grams.keys():
            grams[trigram].add(t[i+N].lower())
        else:
            grams[trigram] = set([t[i+N].lower()])

            
def ngram_generate(grams): #generate text from a dict passed in
    nwords = 0
    start = random.choice([key for key in grams.keys() if key[0].isalpha()])
    res = list(start)
    while(nwords < 100):
        pre = tuple(res[(-1*N):])
        nextword = random.choice(list(grams[pre]))
        res.append(nextword)
        if nextword == '.' or nextword =='?': break
        nwords +=1
    if nwords ==100: res.append('-')
    return " ".join(res).replace(' .', '.').replace(' ,', ',').replace(' ;',';').replace(' ?', '?').replace(' \'', '\'').replace(' !', '!')

def solve_density(grams): #On average, how many words are in the dict for each ngram?
    total = 0.0
    div = 0
    for word in grams.keys():
        if(len(grams[word])>1):
            total += len(grams[word])
            div +=1
    return total/div

In [170]:
#initialize_grams(grams1, 'augustine_full.txt')
#initialize_grams(grams1, 'zhuangzi.txt')
#solve_density(grams1)

In [171]:
class NGramSearchProblem(object):
    def __init__(self, N, grams, magic_words):
        self.N = N
        self.grams = grams
        self.magic_words = magic_words
        self.startActions = None

    # Trivially return 100 if word is magic word, 1 otherwise
    def ngram_cost(self,state):
        if state[-1] in MAGIC_WORDS:
            return 2
        return 1
    
    def ngram_generate_next_words(self, curr_gram):
        pre = tuple(curr_gram[(-1*N):])
        return list(self.grams[pre])
    
    def startState(self):
        valid_start_keys = [key for key in self.grams.keys() if key[0].isalpha()]
        start = random.choice(valid_start_keys) #TODO: initialize first n_gram 
        self.startActions = list(start)
        return list(start)

    def isEnd(self, state):
        return len(state) >= MAX_DIALOGUE_LEN or state[-1][-1] == "." or state[-1][-1] == "?" #max blurb length or last word ends in period
    
    def succAndCost(self, state):
        result = []
        possible_next_words = self.ngram_generate_next_words(state)
        for next_word in possible_next_words:
            next_state = state[:]+[next_word]
            result.append((next_word, next_state, self.ngram_cost(next_state)))
        return result

In [172]:
def backtrackingSearch(problem):
    bestTotalCost = [float('-inf')]
    bestHistory = [None]
    def recurse(state,curr_history,curr_cost):
        if problem.isEnd(state):
            if curr_cost > bestTotalCost[0]:
                bestTotalCost[0], bestHistory[0] = curr_cost, curr_history 
            return
        for action, next_state, cost in problem.succAndCost(state):
            recurse(next_state, curr_history+[action], curr_cost+cost)
    
    recurse(problem.startState(),[],0)
    bestHistory = " ".join(problem.startActions)+" "+" ".join(bestHistory[0])
   # print("<-----------Best History--------->")
    #print(bestHistory)
    #print("<--------------Score------------->")
    #print(bestTotalCost[0])
    return bestHistory

In [173]:
#blurb = backtrackingSearch(NGramSearchProblem(N,grams1,MAGIC_WORDS)) 

In [174]:
def convo(text1, text2, author1, author2):
    initialize_grams(grams1, text1)
    initialize_grams(grams2, text2)
    print('first ngram density: '+ str(solve_density(grams1))+ '\n')
    print('second ngram density: '+ str(solve_density(grams2))+ '\n')
    for i in range(5):
        print(author1 + ':')
        #speech1 = ngram_generate(grams1)
        speech1 = backtrackingSearch(NGramSearchProblem(N,grams1,MAGIC_WORDS))
        print(speech1)
        print('\n')
        #speech2 = ngram_generate(grams2)
        speech2 = backtrackingSearch(NGramSearchProblem(N,grams2,MAGIC_WORDS))
        print(author2 + ':')
        print(speech2)
        print('\n')
        i+=1

In [175]:
convo('augustine_full.txt', 'zhuangzi.txt', 'Augustine', 'Zhuangzi')

first ngram density: 3.4217966453397892

second ngram density: 2.5869829683698295

Augustine:
to whom diurnal thunderbolts were held to pertain .


Zhuangzi:
rest in it - what makes my life a good will make my death also


Augustine:
them is his fate , i would say that the holy spirit is the spirit


Zhuangzi:
of dispute ? 


Augustine:
or diminished by the waxings and wanings of the moon , and the life was


Zhuangzi:
hill , and hide away the hill in a lake , you will fall with


Augustine:
far other from these .


Zhuangzi:
two men looked at one another and come before us , but there was nothing


Augustine:
destruction ? for , as the true god is god , and god with god


Zhuangzi:
really anything to the point or not .




In [176]:
#We need a lot more text for each author, to get higher densities. 
#Right now Zhuangzi is the first 4 books of the inner chapters and Augustine is all of the Confessions
grams1

{('great', 'art', 'thou'): {','},
 ('art', 'thou', ','): {'and', 'o'},
 ('thou', ',', 'o'): {'father',
  'god',
  'great',
  'light',
  'lord',
  'man',
  'most',
  'our',
  'true'},
 (',', 'o', 'lord'): {',', '.', ':', ';', '?', 'god', 'my', 'whom'},
 ('o', 'lord', ','): {'adopted',
  'am',
  'and',
  'are',
  'art',
  'be',
  'behold',
  'blessest',
  'but',
  'do',
  'from',
  'hadst',
  'how',
  'i',
  'in',
  'is',
  'lest',
  'little',
  'lord',
  'memory',
  'most',
  'my',
  'nor',
  'our',
  'pressedst',
  'shalt',
  'taught',
  'that',
  'the',
  'this',
  'thou',
  'to',
  'touched',
  'unchangeably',
  'unto',
  'upon',
  'watched',
  'well',
  'where',
  'which',
  'while',
  'who',
  'will',
  'with'},
 ('lord', ',', 'and'): {'do',
  'greatly',
  'hear',
  'obtaining',
  'reveal',
  'thank',
  'the',
  'with',
  'your'},
 (',', 'and', 'greatly'): {'to', 'weep'},
 ('and', 'greatly', 'to'): {'be'},
 ('greatly', 'to', 'be'): {'desired', 'praised'},
 ('to', 'be', 'praised'): 